In [1]:
import pandas as pd
import ast

In [2]:
emails = pd.read_csv('paul_allen_sent_email_with_questions_v1.csv')
emails['questions'] = emails['questions'].apply(ast.literal_eval)

# Counting how many questions we have

In [3]:
flat_questions = []

for _, row in emails.iterrows():
    flat_questions += row.questions

## Find the easy duplicates

In [9]:
questions_set = {}
duplicates = {}

for i, row in emails.iterrows():
    for question in row.questions:
        if question not in questions_set:
            questions_set[question] = i
        else:
            print(i, question, '\n')
            duplicates[question] = questions_set[question]

14 Who is the recipient of the email? 

20 Who is the recipient of the email? 

25 Who is the recipient of the email? 

34 Who is the recipient of the email? 

44 Who is the recipient of the email? 

52 Who is the recipient of the email? 

53 Who is the sender of the email? 

56 What is Phillip Allen's phone number? 

64 Who is the recipient of the email? 

64 Who is the sender of the email? 

69 What is the URL provided in the email? 

69 Who is the sender of the email? 

90 Who is the sender of the email? 

93 Who is the sender of the email? 

95 Who is the recipient of the email? 

105 Who is the recipient of the email? 

108 Who is the recipient of the email? 

117 Who is the sender of the email? 

119 Who is the recipient of the email? 

131 Who is the recipient of the email? 

132 Who is the recipient of the email? 

133 Who should be added to the distribution list? 

137 Who is the recipient of the email? 

139 Who is the recipient of the email? 

142 Who is the email addressed 

In [10]:
duplicates

{'Who is the recipient of the email?': 9,
 'Who is the sender of the email?': 51,
 "What is Phillip Allen's phone number?": 44,
 'What is the URL provided in the email?': 47,
 'Who should be added to the distribution list?': 93,
 'Who is the email addressed to?': 32,
 "What is the sender's name mentioned in the email?": 137,
 'What is the purpose of the email?': 123,
 'Who sent the email?': 40,
 'Who is being addressed in the email?': 31,
 'Who was the email addressed to?': 46,
 'When will Phillip Allen be out of the office?': 164,
 'What action is being requested in the email?': 145,
 'Who was the recipient of the email?': 243,
 "What is Phillip Allen's cell phone number?": 242,
 'What is the rentroll for this week?': 205,
 'Who is the intended recipient of the email?': 47,
 "What is Phillip Allen's email address?": 123,
 'When is the meeting scheduled?': 149,
 'Who are the recipients of the email?': 220,
 'What style of house is Phillip Allen planning to build?': 341,
 'What are the 

## Find likely duplicates